# Interaction Tracking Analysis

Code to analyze the output from `interactionTracker.js`: process it to determine the likely interactions and reduce the amount of data, then produce some statistics.

In [2]:
import os
import glob

def get_newest_file(directory, startsWith = "interactions_"):
    # Get list of all files in the directory
    files = glob.glob(os.path.join(directory, '*'))
    files = [file.replace("\\", "/") for file in files]
    
    # Check if the directory is empty
    if not files:
        return None

    # Filter for interaction data.
    if startsWith:
        files = [file for file in files if file.split("/")[-1].startswith(startsWith)]
    
    # Get the newest file based on modification time
    newest_file = max(files, key=os.path.getmtime)
    return newest_file

In [14]:
import json

fileDir = 'C:/Users/rafb/AppData/Roaming/Code/User/workspaceStorage/b03f3379a1447d3620f3d4a069f1d983/grandFileNavigator.grandfilenavigator'
filePath = get_newest_file(fileDir)

interactionData = []
with open(filePath) as interactionDataFile:
    for interaction in interactionDataFile.readlines():
        interactionData.append(json.loads(interaction))

print(interactionData)

[{'timeStamp': 1742729006932, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': '', 'sourceRange': '', 'targetRange': '39-73'}, {'timeStamp': 1742729006935, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\helpers\\type_matching.py', 'sourceRange': '39-73', 'targetRange': '40-74'}, {'timeStamp': 1742729006937, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\helpers\\type_matching.py', 'sourceRange': '40-74', 'targetRange': '43-77'}, {'timeStamp': 1742729006940, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\helpers\\type_matching.py', 'sourceRange': '43-77', 'targetRange': '43-78'}, {'timeStamp': 1742729006982, 'interactionType': 'ChangeVisibleRanges', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\helpers\\type_matching.py', 'sourceRange': '43-78', 'targetRange': 

In [16]:
import copy

# Merge scrolls together.
# A scroll is all changes of the visible ranges with less than 250 between them (i.e., covers short interruptions).
def detectScrolling(interactionData):
    maxTimeBetweenChanges = 250 # max time in ms between ChangeVisibleRanges entries to still be considered part of one scroll
    scrollingInteractionData = []

    scrollInteraction = None
    for interaction in interactionData:
        if interaction["interactionType"] != "ChangeVisibleRanges":
            if scrollInteraction is not None:
                scrollingInteractionData.append(scrollInteraction)
                scrollInteraction = None
            scrollingInteractionData.append(interaction)
        elif scrollInteraction is None or interaction["timeStamp"] - scrollInteraction["endTime"] > maxTimeBetweenChanges:
            if scrollInteraction is not None:
                scrollingInteractionData.append(scrollInteraction)
            scrollInteraction = copy.deepcopy(interaction)
            scrollInteraction["interactionType"] = "Scroll"
            scrollInteraction["endTime"] = interaction["timeStamp"]
        else:
            scrollInteraction["endTime"] = interaction["timeStamp"]
            scrollInteraction["targetRange"] = interaction["targetRange"]

    return scrollingInteractionData

In [17]:
# TODO: First process jumps, to prevent them from being treated as scolling data.

scrollingInteractionData = detectScrolling(interactionData)
print(scrollingInteractionData)

[{'timeStamp': 1742729006932, 'interactionType': 'Scroll', 'sourceFilePath': '', 'sourceRange': '', 'targetRange': '61-89', 'endTime': 1742729007373}, {'timeStamp': 1742729007704, 'interactionType': 'Scroll', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\helpers\\type_matching.py', 'sourceRange': '61-89', 'targetRange': '16-50', 'endTime': 1742729009531}, {'timeStamp': 1742729009895, 'interactionType': 'Scroll', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\helpers\\type_matching.py', 'sourceRange': '16-50', 'targetRange': '15-50', 'endTime': 1742729009895}, {'timeStamp': 1742729010196, 'interactionType': 'Scroll', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\helpers\\type_matching.py', 'sourceRange': '15-50', 'targetRange': '15-50', 'endTime': 1742729010203}, {'timeStamp': 1742729016950, 'interactionType': 'Scroll', 'sourceFilePath': 'c:\\Users\\rafb\\source\\repos\\ATAI_Project\\src\\helpers\\type_matching.py', 